In [ ]:
# Data Link-- https://www.kaggle.com/arkhoshghalb/twitter-sentiment-analysis-hatred-speech

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
#load csv file
Train = pd.read_csv("../input/twitter-sentiment-analysis-hatred-speech/train.csv")
data=Train
data.head()

In [ ]:
test_data = pd.read_csv("../input/twitter-sentiment-analysis-hatred-speech/test.csv")
test_data.head()

In [ ]:
data.label.value_counts()

In [ ]:
data.shape

In [ ]:
data['label'].value_counts()

In [ ]:
import seaborn as sns
ax=sns.countplot(data.label)

In [ ]:
# removing usernames from tweet

data['new_tweet'] = data.tweet.str.replace('@user', '')
data.head()

In [ ]:
# removing usernames from test_data

test_data['new_tweet'] = test_data.tweet.str.replace('@user', '')
test_data.head()
ids = test_data['id']

In [ ]:
#Removing Punctuations, Numbers, and Special Characters
#[a-zA-Z] = Any single character in the range a-z or A-Z
# ^ = Start of line 
# $ = End of line 

data['new_tweet'] = data['new_tweet'].str.replace("[^a-zA-Z#]", " ")
data['new_tweet'] = data['new_tweet'].str.replace("#", "")
data.head()


In [ ]:
test_data['new_tweet'] = test_data['new_tweet'].str.replace("[^a-zA-Z#]", " ")
test_data['new_tweet'] = test_data['new_tweet'].str.replace("#", "")
test_data.head()

In [ ]:
# get most common words in training dataset
from collections import Counter 
all_words = []
for line in list(data['new_tweet']):
    words = line.split()
    for word in words:
        all_words.append(word.lower())
    
    
a=Counter(all_words).most_common(10)
a

In [ ]:
#tokenization
data['new_tweet'] = data['new_tweet'].apply(lambda x: x.split())
data.head()

In [ ]:
test_data['new_tweet'] = test_data['new_tweet'].apply(lambda x: x.split())
test_data.head()

In [ ]:
#stemmer
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

data['new_tweet']= data['new_tweet'].apply(lambda x: [stemmer.stem(i) for i in x])
data.head()

In [ ]:
test_data['new_tweet']= test_data['new_tweet'].apply(lambda x: [stemmer.stem(i) for i in x])
test_data.head()

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
from nltk.corpus import stopwords

from nltk.tokenize import word_tokenize

#stopwords = set(stopwords.words('english'))
stopwords = nltk.corpus.stopwords.words('english')



In [ ]:
newStopWords = ['u','go','got','via','or','ur','us','in','i','let','the','to','is','amp','make','one','day','days','get']
stopwords.extend(newStopWords)


In [ ]:
import string 

def process(text):
    # Check characters to see if they are in punctuation
    nopunc = set(char for char in list(text) if char not in string.punctuation)
    # Join the characters to form the string.
    nopunc = " ".join(nopunc)
    # remove any stopwords if present
    return [word for word in nopunc.lower().split() if word.lower() not in stopwords]
    

In [ ]:
data['new_tweet'] = data['new_tweet'].apply(process) 
data.head()

In [ ]:
# test_data
test_data['new_tweet'] = test_data['new_tweet'].apply(process) 
test_data.head()

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt


# split sentences to get individual words
words = []
for line in data['new_tweet']: 
    words.extend(line)
    
# create a word frequency dictionary
wordfreq = Counter(words)
# draw a Word Cloud with word frequencies
wordcloud = WordCloud(
    background_color='white',
    max_words=2000,
    stopwords=stopwords
   ).generate_from_frequencies(wordfreq)
plt.figure(figsize=(10,9))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
def string (text):
    to_return=""
    for i in list(text):
        to_return += str(i) + " "
    to_return = to_return[:-1]
    
    return to_return
    
       
data['new_tweet'] = data['new_tweet'].apply(string)
data.head()  

In [ ]:
test_data['new_tweet'] = test_data['new_tweet'].apply(string)
test_data.head()

In [ ]:
positive = [r for r in data['new_tweet'][data['label']==0]]
pos = ''.join(positive)

# draw a Word Cloud with word frequencies
wordcloud = WordCloud(
    background_color='white',
    max_words=2000,
    stopwords=stopwords
   ).generate(pos)
plt.figure(figsize=(10,9))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
negative = [r for r in data['new_tweet'][data['label']==1]]
neg = ''.join(negative)

# draw a Word Cloud with word frequencies
wordcloud = WordCloud(
    background_color='black',
    max_words=2000,
    stopwords=stopwords
   ).generate(neg)
plt.figure(figsize=(10,9))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

### Bag of words 

In [ ]:
data.drop(["id","tweet" ],axis=1,inplace=True)
data.head()

In [ ]:
test_data.drop(["id","tweet" ],axis=1,inplace=True)
test_data.head()

In [ ]:
#Split data into training and testing sets 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data["new_tweet"], 
                                                    data["label"], test_size = 0.2, random_state = 42)

print("training set :",x_train.shape,y_train.shape)
print("testing set :",x_test.shape,y_test.shape)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
count_vect = CountVectorizer(stop_words='english')
transformer = TfidfTransformer(norm='l2',sublinear_tf=True)


In [ ]:
test_x = test_data['new_tweet']
test_x

In [ ]:
x_train_counts = count_vect.fit_transform(x_train)
x_train_tfidf = transformer.fit_transform(x_train_counts)

print(x_train_counts.shape)
print(x_train_tfidf.shape)

In [ ]:
test_x_counts = count_vect.transform(test_x)
test_x_tfidf = transformer.transform(test_x_counts)

print(test_x_counts.shape)
print(test_x_tfidf.shape)

In [ ]:
x_test_counts = count_vect.transform(x_test)
x_test_tfidf = transformer.transform(x_test_counts)

print(x_test_counts.shape)
print(x_test_tfidf.shape)

### Model building

#### Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=200)
model.fit(x_train_tfidf,y_train)

In [ ]:
predictions = model.predict(x_test_tfidf)

In [ ]:
submission  = model.predict(test_x_tfidf)

In [ ]:
#Accuracy_score
from sklearn.metrics import accuracy_score
accuracy_score(y_test,predictions)*100

In [ ]:
from sklearn.metrics import confusion_matrix,f1_score
confusion_matrix(y_test,predictions)

In [ ]:
#f1-score
f1_score(y_test,predictions)

In [ ]:
from __future__ import print_function


# Special END separator
END = '0e8ed89a-47ba-4cdb-938e-b8af8e084d5c'

# Text attributes
ALL_OFF = '\033[0m'
BOLD = '\033[1m'
UNDERSCORE = '\033[4m'
BLINK = '\033[5m'
REVERSE = '\033[7m'
CONCEALED = '\033[7m'

# Foreground colors
FG_BLACK = '\033[30m'
FG_RED = '\033[31m'
FG_GREEN = '\033[32m'
FG_YELLOW = '\033[33m'
FG_BLUE = '\033[34m'
FG_MAGENTA = '\033[35m'
FG_CYAN = '\033[36m'
FG_WHITE = '\033[37m'

# Background colors
BG_BLACK = '\033[40m'
BG_RED = '\033[41m'
BG_GREEN = '\033[42m'
BG_YELLOW = '\033[43m'
BG_BLUE = '\033[44m'
BG_MAGENTA = '\033[45m'
BG_CYAN = '\033[46m'
BG_WHITE = '\033[47m'


class pretty_output():
    '''
    Context manager for pretty terminal prints
    '''

    def __init__(self, *attr):
        self.attributes = attr

    def __enter__(self):
        return self

    def __exit__(self, type, value, traceback):
        pass

    def write(self, msg):
        style = ''.join(self.attributes)
        print('{}{}{}'.format(style, msg.replace(END, ALL_OFF + style), ALL_OFF))


if __name__ == '__main__':

    with pretty_output(FG_RED) as out:
        out.write('This is a test in RED')

    with pretty_output(FG_BLUE) as out:
        out.write('This is a test in BLUE')

    with pretty_output(BOLD, FG_GREEN) as out:
        out.write('This is a bold text in green')

    with pretty_output(BOLD, BG_GREEN) as out:
        out.write('This is a text with green background')

    with pretty_output(FG_GREEN) as out:
        out.write('This is a green text with ' + BOLD + 'bold' + END + ' text included')

    with pretty_output() as out:
        out.write(BOLD + 'Use this' + END + ' even with ' + BOLD + FG_RED + 'no parameters' + END + ' in the with statement')

In [ ]:
from sklearn.metrics import classification_report
df = pd.DataFrame(classification_report(predictions, 
                                        y_test, digits=2,
                                        output_dict=True)).T

with pretty_output(BOLD, FG_GREEN) as out:
    out.write('                 RANDOM FOREST                 ')
df.style.background_gradient(cmap='viridis',
                             subset=pd.IndexSlice['0':'1', :'f1-score'],)


In [ ]:
def color_negative_red(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
    color = 'red' if val == 'NEG' else 'green'
    return 'color: %s' % color

In [ ]:
tweets = pd.DataFrame(columns=['Tweets','Prediction','Label'])
tweets['Tweets'] = x_test
tweets['Prediction'] = predictions
tweets['Label'] = y_test
tweets.replace([0,1],['POS', 'NEG'], inplace=True)
with pretty_output(BOLD, FG_GREEN) as out:
    out.write('                 RANDOM FOREST Tweets Prediction                ')
(tweets.sample(10).style.applymap(color_negative_red, subset=['Prediction', 'Label']))

In [ ]:
predictions

In [ ]:
test_data.head()

In [ ]:
submission_df = pd.DataFrame(columns=['id', 'label'])
submission_df['id'] = ids
submission_df['label'] = submission
submission_df.head(5)

In [ ]:
submission_df.to_csv('submission.csv',index=False)

### Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
logmodel = LogisticRegression(random_state=400 )
logmodel.fit(x_train_tfidf,y_train)

In [ ]:
log_predictions = logmodel.predict(x_test_tfidf)

In [ ]:
from sklearn.metrics import confusion_matrix,f1_score
confusion_matrix(y_test,log_predictions)

In [ ]:
f1_score(y_test,log_predictions)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,log_predictions)*100

In [ ]:
submission  = model.predict(test_x_tfidf)
submission_df = pd.DataFrame(columns=['id', 'label'])
submission_df['id'] = ids
submission_df['label'] = submission
submission_df.to_csv('submission-log.csv',index=False)
submission_df.head(5)

### Gradient Boosting Regressor

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

alg= GradientBoostingRegressor(n_estimators= 550, learning_rate= 0.1, max_depth= 3)
alg.fit(x_train_tfidf,y_train)

In [ ]:
alg_predictions = logmodel.predict(x_test_tfidf)

In [ ]:
from sklearn.metrics import confusion_matrix,f1_score
confusion_matrix(y_test,alg_predictions)

In [ ]:
f1_score(y_test,alg_predictions)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,alg_predictions)*100

In [ ]:
import seaborn as sns

In [ ]:
models = ["Naive Bayes", 'Random Forest', 'XGBoost']
acc = [94, 96, 95]
plt.style.use('ggplot')
plt.figure(figsize=(6, 4))
sns.barplot(models, acc, );
# Get current axis on current figure
ax = plt.gca()

# ylim max value to be set
y_max = 110 
ax.set_ylim([0, y_max])

# Iterate through the list of axes' patches
for p in ax.patches:
    ax.text(p.get_x() + p.get_width()/2., p.get_height(), '%d' % int(p.get_height()), 
            fontsize=12, color='red', ha='center', va='bottom')
plt.title('Accuracy of classifiers')
plt.show()

In [ ]:
submission  = model.predict(test_x_tfidf)
submission_df = pd.DataFrame(columns=['id', 'label'])
submission_df['id'] = ids
submission_df['label'] = submission
submission_df.to_csv('submission_gb.csv',index=False)
submission_df.head(5)